In [20]:
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import numpy as np
import cv2
from moviepy.editor import VideoFileClip
import math
import os

# Global parameters
# Gaussian smooting
kernel_size = 3

# Canny Edge Detector
low_threshold = 50
high_threshold = 150

# Refion of interest vertices
# we want to trapezoid shape, with bottom edge at the bottom of the image
trap_bottom_width = 0.85 # width of bottom edge of trapezoid, expressed as percentage of image width
trap_top_width = 0.07    # ditto for top edge of trapezoid
trap_height = 0.4        # height of the trapezoid expressed as percentage of image height

# Hough Transfer
rho = 2 # distance resolution in pixels of Hough grid
theta = 1*np.pi/180  # angular resolution in radians of the Hough grid
threshold = 15    # minimun number of votes(intersections in Hough grid cell)
min_line_length = 10    # minmun number of pixels making up a line
max_line_gap = 20      # maxmum gap in pixels between connectable line segments


# helper functions 
def grayscale(img):
    """Applies the Grayscale transform
	This will return an image with only one color channel
	but NOTE: to see the returned image as grayscale
	you should call plt.imshow(gray, cmap='gray')
    """
    return cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)

def canny(img, low_threshold, high_threshold):
    """Applies the Canny transform"""
    return cv2.Canny(img, low_threshold, high_threshold)

def gaussian_blur(img, kernel_size):
    """Applies a Gaussian Noise kernel"""
    return cv2.GaussianBlur(img, (kernel_size, kernel_size), 0)

def region_of_interest(img,vertices):
    '''
    Applies an image mask.
    
    only keeps the region of the image defined by the polygon formed from `vertices`. The rest 
    of the image is set to black
    '''
    #defining a blank mask to start with
    mask = np.zeros_like(img)   
	
	#defining a 3 channel or 1 channel color to fill the mask with depending on the input image
    if len(img.shape) > 2:
        channel_count = img.shape[2]  # i.e. 3 or 4 depending on your image
        ignore_mask_color = (255,) * channel_count
    else:
        ignore_mask_color = 255
		
	#filling pixels inside the polygon defined by "vertices" with the fill color	
    cv2.fillPoly(mask, vertices, ignore_mask_color)
	
	#returning the image only where mask pixels are nonzero
    masked_image = cv2.bitwise_and(img, mask)
    return masked_image

def draw_lines(img,lines,color=[255,0,0],thickness = 10):
    # in case of error, don't draw the lines
    if lines is None:
        return 
    if len(lines) == 0:
        return 
    draw_right = True
    draw_left = True
    
    # Find slopes of all lines
    # But only care about lines where abs(slope) > slope_threshold

    slope_threshold = 0.5
    slopes = []
    new_lines = []
    for line in lines:
        x1,y1,x2,y2 = line[0] # line= [[x1,y1,x2,y2]]
        
        # calcuate slope 
        if x2 - x1 == 0.:# corner casem avoiding division by 0
            slope = 999.  # practically infinite slope
            
        else:
            slope = (y2-y1)/(x2-x1)
            
        # Filter lines based on slope
        if abs(slope) > slope_threshold:
            slopes.append(slope)
            new_lines.append(line)
    lines = new_lines
    
    # Split lines into right_lines and left_lines, representing the right and left lane lines
	# Right/left lane lines must have positive/negative slope, and be on the right/left half of the image
    right_lines = []
    left_lines = []
    for i, line in enumerate(lines):
        x1,y1,x2,y2 = line[0]
        img_x_center = img.shape[1]/2   # x coordinate of center of image
        if slopes[i] > 0 and x1 > img_x_center and x2 > img_x_center:
            right_lines.append(line)
        elif slopes[i] < 0 and x1 < img_x_center and x2< img_x_center:
            left_lines.append(line)
            
    # Run linear regression to find best fit line for right and left lane lines
    right_lines_x = []
    right_lines_y = []
    
    for line in right_lines:
        x1,y1,x2,y2 = line[0]
        
        right_lines_x.append(x1)
        right_lines_x.append(x2)
        
        right_lines_y.append(y1)
        right_lines_y.append(y2)
        
    if len(right_lines_x) > 0:
        right_m, right_b = np.polyfit(right_lines_x, right_lines_y,1)  # y = m*x + b
    else:
        right_m, right_b = 1,1
        draw_right = False
            
    # Left lane lines
    left_lines_x = []
    left_lines_y = []
	
    for line in left_lines:
        x1, y1, x2, y2 = line[0]
		
        left_lines_x.append(x1)
        left_lines_x.append(x2)
		
        left_lines_y.append(y1)
        left_lines_y.append(y2)
		
    if len(left_lines_x) > 0:
        left_m, left_b = np.polyfit(left_lines_x, left_lines_y, 1)  # y = m*x + b
    else:
        left_m, left_b = 1, 1
        draw_left = False
     
    # Find 2 end points for right and left lines, used for drawing the line
    # y = m*x + b  --> x = (y-b)/m
    y1 = img.shape[0]
    y2 = img.shape[0] * (1-trap_height)
    
    right_x1 = (y1 - right_b)/right_m
    right_x2 = (y2 - right_b)/right_m
    
    left_x1 = (y1 - left_b)/left_m
    left_x2 = (y2 - left_b)/left_m
    
    # Convert calculated end points from float to int
    y1 = int(y1)
    y2 = int(y2)
    right_x1 = int(right_x1)
    right_x2 = int(right_x2)
    left_x1 = int(left_x1)
    left_x2 = int(left_x2)
    
    # Draw the right and left lines on image
    if draw_right:
        cv2.line(img,(right_x1,y1),(right_x2,y2),color,thickness)
        
    if draw_left:
        cv2.line(img,(left_x1,y1),(left_x2,y2), color, thickness)
        
def hough_lines(img,rho,theta,threshold,min_line_len, max_line_gap):
    '''
    `img` should be the output of a Canny transform
    
    returns an image with hough lines drawn
    '''
    lines = cv2.HoughLinesP(img,rho,theta,threshold,np.array([]),minLineLength=min_line_len,maxLineGap=max_line_gap)
    line_img = np.zeros((*img.shape,3),dtype=np.uint8)   # 3 channel RGB image
    draw_lines(line_img, lines)
    return line_img

def weighted_img(img, initial_img, α=0.8, β=1., λ=0.):
    """
    `img` is the output of the hough_lines(), An image with lines drawn on it.
    Should be a blank image (all black) with lines drawn on it.

    `initial_img` should be the image before any processing.
    
    The result image is computed as follows:

    initial_img * α + img * β + λ
    NOTE: initial_img and img must be the same shape!
    """
    return cv2.addWeighted(initial_img, α, img, β, λ)


def filter_color(image):
    '''
    Filter white pixels
    '''
    white_threshold = 200
    lower_white = np.array([white_threshold, white_threshold,white_threshold])
    upper_white = np.array([255,255,255])
    white_mask = cv2.inRange(image, lower_white, upper_white)
    white_image = cv2.bitwise_and(image, image, mask=white_mask)
    
    # Filter yellow pixels
    hsv = cv2.cvtColor(image,cv2.COLOR_BGR2HSV)
    lower_yellow = np.array([90,100,100])
    upper_yellow = np.array([100,255,255])
    yellow_mask = cv2.inRange(hsv, lower_yellow, upper_yellow)
    yellow_image = cv2.bitwise_and(image,image,mask=yellow_mask)
    
    # Combine the two above images
    image2 = cv2.addWeighted(white_image, 1., yellow_image,1.,0.)
    
    return image2

def annotate_image_array(image_in):
    '''Given an image Numpy array, return the annotated image as Numpy array'''
    # pnly keep white and yellow pixels in the image, all other pixels become black
    image = filter_color(image_in)
    
    
    # Read in and grayscale the image
    gray = grayscale(image)
    
    # Apply Gaussian blur
    blur_gray = gaussian_blur(gray, kernel_size)
    
    
    # Apply Canny edge detector
    edges = canny(blur_gray, low_threshold, high_threshold)
    
    # Create masked edges using trapezoid-shaped region-of-interest
    imshape = image.shape
    vertices = np.array([[\
                         ((imshape[1]*(1-trap_bottom_width)) // 2, imshape[0]),\
                         ((imshape[1]*(1-trap_top_width))//2,imshape[0] - imshape[0]*trap_height),\
                         (imshape[1] - (imshape[1]*(1-trap_top_width))//2,imshape[0]*trap_height),\
                         (imshape[1] - (imshape[1] *(1- trap_bottom_width))//2, imshape[0])]]\
                         ,dtype=np.int32)
    masked_edges = region_of_interest(edges, vertices)
    
    # Run Hough on edge detected image
    line_image = hough_lines(masked_edges, rho, theta, threshold, min_line_length, max_line_gap)
    
    # Draw lane lines on the original image
    initial_image = image_in.astype('uint8')
    annotated_image = weighted_img(line_image, initial_image)
    
    return annotated_image

def annotate_image(input_file, out_file):
    '''Given input_file image, save annotated image to output_file'''
    annotated_image = annotate_image_array(mpimg.imread(input_file))
    plt.imsave(output_file, annotated_image)
    
def annotate_video(input_file, out_file):
    '''Given input_file image, save annotated image to output_file'''
    video = VideoFileClip(input_file)
    annotated_video = video.fl_image(annotate_image_array)
    annotated_video.write_videofile(out_file,audio=False)
    
# End helper functions       

In [23]:
# main script
if __name__ == '__main__':
    from optparse import OptionParser
    
    # Configure command line options
    parser = OptionParser()
    parser.add_option("-i", "--input_file", dest="input_file", help="Input video/image file")
    parser.add_option("-o", "--output_file", dest="output_file",help="Output (destination) video/image file")
    parser.add_option("-I", "--image_only",action="store_true", dest="image_only", default=False,
                      help="Annotate image (defaults to annotating video)")
    
    
    # Get and parse command line options
    options, args = parser.parse_args()
    
    input_file = options.input_file
    output_file = options.output_file
    image_only = options.image_only
    
    if image_only:
        annotate_image(input_file, output_file)
        
    else:
        annotate_video(input_file, output_file)

Usage: ipykernel_launcher.py [options]

ipykernel_launcher.py: error: no such option: -f
ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "D:\Self Learning\Anaconda3\envs\tf2.0.0rc1\lib\optparse.py", line 1387, in parse_args
    stop = self._process_args(largs, rargs, values)
  File "D:\Self Learning\Anaconda3\envs\tf2.0.0rc1\lib\optparse.py", line 1431, in _process_args
    self._process_short_opts(rargs, values)
  File "D:\Self Learning\Anaconda3\envs\tf2.0.0rc1\lib\optparse.py", line 1513, in _process_short_opts
    raise BadOptionError(opt)
optparse.BadOptionError: no such option: -f

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "D:\Self Learning\Anaconda3\envs\tf2.0.0rc1\lib\site-packages\IPython\core\interactiveshell.py", line 3331, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-23-46f85d1394f0>", line 14, in <module>
    options, args = parser.parse_args()
  File "D:\Self Learning\Anaconda3\envs\tf2.0.0rc1\lib\optparse.py", line 1389, in parse_args
    self.error(str

SystemExit: 2